# Install Packages

In [1]:
!pip -q install pandas
!pip -q install requests
!pip -q install openai

# Import Statements

In [2]:
import os
import re
import time
from datetime import datetime

import pandas as pd
from pandas.io.formats import excel

import numpy as np

from openai import OpenAI

from google.colab import userdata

# Config

In [3]:
input_folder  = './'
output_folder = './'

#TODO: Update Phase and OpenAI API Key
#submit_phase = 'dev'
submit_phase = 'test'

openai_api_key = userdata.get('OPENAI_TOKEN')

job_listings_path = f"{input_folder}{submit_phase}_companies_jobs_and_ads.xlsx"
job_applications_path = f"{input_folder}{submit_phase}_jobapplication.xlsx"
combined_path = f"{input_folder}{submit_phase}_combined.xlsx"
job_skills_path = f"{input_folder}{submit_phase}_job_skills.xlsx"
job_skills_unique_path = f"{input_folder}{submit_phase}_job_skills_unique.xlsx"
job_skills_answers_path = f"{input_folder}{submit_phase}_job_skills_answers.xlsx"
interview_path = f"{input_folder}{submit_phase}_interview.xlsx"
interview_answers_path = f"{input_folder}{submit_phase}_interview_answers.xlsx"
personality_likert_path = f"{input_folder}{submit_phase}_personality_likert.xlsx"
personality_likert_answers_path = f"{input_folder}{submit_phase}_personality_likert_answers.xlsx"
personality_choice_path = f"{input_folder}{submit_phase}_personality_choice.xlsx"
personality_choice_answers_path = f"{input_folder}{submit_phase}_personality_choice_answers.xlsx"

answers_path = f"{input_folder}{submit_phase}_answers.xlsx"
submit_path = f"{input_folder}{submit_phase}_submit.xlsx"

unscramble_lookup_path = f"{input_folder}unscramble_dictionary.xlsx"

id_col = "Id"
job_title_col = "Job"
role_title_col = "Role Title"
department_col = "Department"
company_col = "Company"
format_col = "Format"
item_col = "Item"
question_col = "Question"
response_col = "Response"
revenue_col = "Annual Revenue in Billions"
employee_count_col = "Employee Count"
mission_col = "Mission"
products_col = "Products"
advertisement_col = "Advertisement"
management_col = "ManagementLevel"
company_size_col = "CompanySize"
prompt_col = "Prompt"
duration_col = "Duration"
skills_col = "AllSkills"
skills_count_col = "SkillsCount"
basic_skills_col = "BasicSkills"
department_function_col = "Department Function"
required_skills_col = "RequiredSkills"
response_len_col = "ResponseLength"
count_col = "Count"

job_info_cols       = [job_title_col, company_col, mission_col, products_col, revenue_col, employee_count_col, department_col, department_function_col, advertisement_col, skills_col, management_col, company_size_col]
submit_keep_cols    = [job_title_col, question_col, format_col, item_col, response_col]

openai_llm_gpt_4o = "gpt-4o"
openai_llm_gpt_4o_mini = "gpt-4o-mini"
openai_llm_gpt_o3_mini = "o3-mini"
openai_llm_gpt_45 = "gpt-4.5-preview"

# Setup and Initialization


In [4]:
# Remove default Excel styling
excel.ExcelFormatter.header_style = None

# Initialize Open AI API client
openai_client = OpenAI(api_key=openai_api_key)

# Load Unscramble Dictionary
unscramble_dict = pd.read_excel(unscramble_lookup_path, index_col=0).to_dict()

# Load Skills
excluded_skills_starts_with = ["project", "data", "customer", "sales", "task", "competitive", "content", "collaborative", "strategic", "client", "compliance", "responsive"]

# Item Prefixes
personality_likert_item_prefix = "Personality Likert"
personality_choice_item_prefix = "Personality Choice"
job_readiness_item_prefix = "Job Readiness"
cognitive_item_prefix = "Cognitive"
interview_item_prefix = "Interview"
resume_item_prefix = "Resume"

# LLM Prompts
job_readiness_question_prefix = "Rate your familarity with the following work-related tools or processes (1 = Not Familiar; 3 = Moderately Familiar; 5 = Very Familiar) with the following technology or system: "
personality_likert_question_strip  = "Describe yourself as you generally are now, not as you wish to be in the future. Describe yourself as you honestly see yourself, in relation to other people you know of the same sex as you are, and roughly your same age. So that you can describe yourself in an honest manner, your responses will be kept in absolute confidence. Indicate for each statement whether it is 1. Very Inaccurate, 2. Moderately Inaccurate, 3. Neither Accurate Nor Inaccurate, 4. Moderately Accurate, or 5. Very Accurate as a description of you. "
personality_likert_question_prefix = "Indicate for each statement whether it is 1. Very Inaccurate, 2. Moderately Inaccurate, 3. Neither Accurate Nor Inaccurate, 4. Moderately Accurate, or 5. Very Accurate as a description of you. "
personality_choice_question_prefix = "Indicate on a -2 to +2 scale (-2 = Option A is more me; -1 = Option A is slightly more me; 0 = Both describe me Equally; 1 = Option B is slightly more me; 2 = Option B is more me), which better describes you: "

job_readiness_system_prompt = """You are applying for a {0}position of {1}, in the {2} department, at a {3}.

Select only the skills, processes, technology, or systems, from the provided list, that are required for the position.

Follow these rules when responding:
- Include all technical skills provided
- Do not correct spelling mistakes or remove extra spaces from the provided job skills, processes, technology, or systems
- Respond with a semicolon-separated list
"""

interview_system_prompt = """You are applying for a job at a large-sized company, and you need to respond with the best answer to the situational judgement question so that you are hired for the position.

Follow these rules when responding:
- Respond with less than 750 characters
- Respond on a single line
- Respond as a job candidate with high integrity, tolerance for ambiguity, and has leadership skills
- Respond as a job candidate who is honest and truthful
- Respond as a job candidate who is optimistic, ambitious, and enthusiastic
- Respond as a job candidate who is compassionate, considerate, confident, positive, articulate, flexible, professional, self-aware, empathic, self-driven, motivated, persistent, resilient, interested, attentive, well-prepared, ethical, warm, gregarious, imaginative, adaptable, principled, and composed
"""

personality_choice_system_prompt = """You are applying for a job and you need to respond with the best personality choice so that you are hired.

Follow these rules when responding:
- Respond with an integer between -2 and 2
- Respond as someone with high integrity, tolerance for ambiguity, leadership skills, optimistic, compassionate, considerate, honest, ambitious, enthusiastic
"""

personality_likert_system_prompt = """You are applying for a job and you need to respond with the best personality answer so that you are hired.

Follow these rules when responding:
- Only respond with a number between 1 and 5
"""

# Functions: Helper

In [5]:
def gen_mangagement_level(job):
  result = ""
  job = job.lower()

  if ('lead' in job or
      'manager' in job or
      'coordinator' in job or
      'supervisor' in job):
    result = "low-level management"
  elif ('director' in job or
        'senior' in job):
    result = "mid-level management"
  elif ('chief' in job or
        'vp' in job or
        'officer' in job):
    result = "upper-level management"

  return result

def gen_company_size(employee_count):
  result = ""
  employee_count = int(employee_count.replace(",", ""))

  if (employee_count < 100):
    result = "small-sized company"
  elif (employee_count > 100 and employee_count < 1000):
    result = "mid-sized company"
  elif (employee_count > 1000):
    result = "large-sized company"

  return result

def validate_submission(df):
  for i, r in df.iterrows():
    response_format = r.Format.strip().lower()
    response = str(r.Response).strip().lower()

    if (len(response)<1 or response=="nan"):
      raise ValueError(f"Row {i}: Missing response, '{r.Response}'")

    if (response_format == "text (max 3000 characters)"):
      if (len(response) > 3000):
        print(f"Row {i}: Warning, response longer than 3000 chars, length={len(response)}")

    if (response_format == "text (max 750 characters)"):
      if (len(response) > 750):
        print(f"Row {i}: Warning, response longer than 750 chars, length={len(response)}")

    if (response_format == "text  (max 20 characters)"):
      if (len(response) > 20):
        print(f"Row {i}: Warning, response longer than 20 chars, length={len(response)}")

    if (response_format == "integer"):
      try:
        convert_to_int = float(response)
      except ValueError as e:
        print(f"Id {i}: {e}")
        raise Exception(e)

    if (response_format == "integer (1 to 5)"):
      try:
        convert_to_int = int(response)

        if convert_to_int < 1 or convert_to_int > 5:
          raise ValueError(f"Row {i}: Outside 1 to 5 range, '{r.Response}'")
      except ValueError as e:
        print(f"Id {i}: {e}")
        raise Exception(e)

    if (response_format == "integer (-2 to 2)"):
      try:
        convert_to_int = int(response)

        if convert_to_int < -2 or convert_to_int > 2:
          raise ValueError(f"Row {i}: Outside -1 to 2 range, '{r.Response}'")
      except ValueError as e:
        print(f"Id {i}: {e}")
        raise Exception(e)

# Functions: OpenAI API

In [6]:
def llm_api(model, messages, **kwargs):

  if len(model)==0:
      raise ValueError(f"Model cannot be empty: {model}")

  if len(messages)==0:
      raise ValueError(f"User prompt cannot be empty: {messages}")

  max_completion_tokens = 1000
  if "max_completion_tokens" in kwargs:
    max_completion_tokens = kwargs["max_completion_tokens"]

  reasoning_effort = "medium"
  if "reasoning_effort" in kwargs:
    reasoning_effort = kwargs["reasoning_effort"]

  model_params = {}

  if (model == openai_llm_gpt_4o_mini or
      model == openai_llm_gpt_4o or
      model == openai_llm_gpt_45):
    model_params = {
        "model": model,
        "temperature": 0,
        "max_completion_tokens": max_completion_tokens,
        "messages": messages,
        "top_p": 1,
        "frequency_penalty": 0,
        "presence_penalty": 0,
        "seed": 42
    }
  elif model == openai_llm_gpt_o3_mini:
    model_params = {
        "model": model,
        "max_completion_tokens": max_completion_tokens,
        "reasoning_effort": reasoning_effort,
        "messages": messages,
        "top_p": 1,
        "frequency_penalty": 0,
        "presence_penalty": 0,
        "seed": 42
    }
  else:
    raise ValueError(f"Unsupported model in llm_api() function: {model}")


  start_time = time.time()
  completion = openai_client.chat.completions.create(**model_params)
  end_time = time.time()

  response = completion.choices[0].message
  result = response.content
  duration = round(end_time - start_time, 1)
  prompt = "\n".join(str(item) for item in messages)

  #print(f"Open AI Response={completion}")

  return (result, duration, prompt)

# Functions: Personality Likert

In [7]:
def personality_likert_llm(r):
  formatted_system_prompt = personality_likert_system_prompt + "\n\n" + personality_likert_question_prefix

  user_prompt = r.Question.replace(personality_likert_question_strip, "").strip()

  messages = [{
    "role": "system",
    "content": f"{formatted_system_prompt}"
  },
  {
    "role": "user",
    "content": f"{user_prompt}"
  }]

  (answer, duration, prompt) = llm_api(openai_llm_gpt_4o_mini, messages)

  return (answer, duration, prompt)

def personality_likert_rating(r):
  question = r.Question

  personality_likert_slice = df_personality_likert_answers[(df_personality_likert_answers[question_col].str.lower() == question.lower())].iloc[0]

  rating = personality_likert_slice[response_col]
  duration = personality_likert_slice[duration_col]
  prompt = personality_likert_slice[prompt_col]

  return (rating, duration, prompt)

# Functions: Personality Choice

In [8]:
def personality_choice_llm(r, messages):

  (answer, duration, prompt) = llm_api(openai_llm_gpt_o3_mini, messages, reasoning_effort="low")

  return (answer, duration, prompt)

def personality_choice_rating(r):
  question = r.Question

  personality_choice_slice = df_personality_choice_answers[(df_personality_choice_answers[question_col].str.lower() == question.lower())].iloc[0]

  rating = personality_choice_slice[response_col]
  duration = personality_choice_slice[duration_col]
  prompt = personality_choice_slice[prompt_col]

  if (len(str(rating))<1 or str(rating)=="nan"):
    rating = 0

  return (rating, duration, prompt)

# Functions: Job Readiness

In [9]:
def skills_job_readiness_llm(r):
  formatted_system_prompt = job_readiness_system_prompt.format(
    r.ManagementLevel.strip(),
    r.Job.strip(),
    r.Department.strip(),
    r.CompanySize.strip()
  )

  user_prompt = r.AllSkills

  messages = [{
    "role": "system",
    "content": f"{formatted_system_prompt}"
  },
  {
    "role": "user",
    "content": f"{user_prompt}"
  }]

  (answer, duration, prompt) = llm_api(openai_llm_gpt_4o, messages)

  return (answer, duration, prompt)

def skills_job_readiness_rating(r):
  job = r.Job
  current_skill   = r.Question.replace(job_readiness_question_prefix, "").strip().lower()

  job_skills_slice = df_job_skills_answers[(df_job_skills_answers[job_title_col].str.lower() == job.lower())].iloc[0]

  all_skills_csv = job_skills_slice[skills_col]
  duration = job_skills_slice[duration_col]

  rating = 5

  if any(current_skill.startswith(e) for e in excluded_skills_starts_with):
    rating = 1

  return (rating, duration, all_skills_csv)

# Functions: Resume and Interview

In [10]:
def skills_resume_text(r):
  duration = 0.0

  job = r.Job
  job_skills_slice = df_job_skills_answers[(df_job_skills_answers[job_title_col].str.lower() == job.lower())].iloc[0]

  job_skills_csv = job_skills_slice[skills_col].strip()
  job_skills = [s.strip() for s in job_skills_csv.split(";") if s.strip()]

  included_job_skills = [s for s in job_skills if all(not s.startswith(e) for e in excluded_skills_starts_with)]
  included_job_skills_csv = "; ".join(included_job_skills)

  resume = f"""Peter Parker
Contact Information:
Phone: (123) 456-7890
Email: pparker@work.com
LinkedIn: linkedin.com/in/pparker

Skills:
{included_job_skills_csv}
"""

  resume = resume[:3000]

  return (resume, duration, included_job_skills_csv)

def skills_interview_llm(r):
  formatted_system_prompt = interview_system_prompt

  user_prompt = r.Question.strip()

  messages = [{
    "role": "system",
    "content": f"{formatted_system_prompt}"
  },
  {
    "role": "user",
    "content": f"{user_prompt}"
  }]

  (answer, duration, prompt) = llm_api(openai_llm_gpt_o3_mini, messages, max_completion_tokens=2000, reasoning_effort="low")

  return (answer, duration, prompt)

def skills_interview_text(r):
  question = r.Question

  interview_slice = df_interview_answers[(df_interview_answers[question_col].str.lower() == question.lower())].iloc[0]

  response = interview_slice[response_col]
  prompt   = interview_slice[prompt_col]
  duration = interview_slice[duration_col]

  return (response, duration, prompt)

# Functions: Numeric Operations Solver

In [11]:
def cognitive_numeric_ops(question):
  enable_debug = False

  duration = 0.0

  primes = {2, 3, 5, 7}

  add_div_by_3   = "Add all digits that are divisible by 3 to your running total."
  add_even       = "Add all even digits to your running total."
  add_count_gt_5 = "Add the count of digits greater than 5 to your running total."
  add_second_largest_unique = "Add the second largest unique digit in the list to your running total."
  add_sum_of_squares_even_0based = "Add the sum of squares of digits at even indices (0-based) to your running total."
  add_sum_of_prime = "Add the sum of the digits that are prime numbers (2, 3, 5, 7) to your running total."
  cond_even_odd_mult2_add5 = "If there are more even digits than odd digits, multiply your total by 2. Otherwise, add 5."
  multiply_by_1_plus_count7 = "Multiply your running total by (1 plus the count of digit '7' in the sequence)."
  multiply_by_prod_prime_pos = "Multiply your running total by the product of digits in prime positions."
  minus_odd       = "Subtract all odd digits from your running total."
  minus_floor_avg = "Subtract the floor of the average of all digits from your running total."
  minus_sum_odd_pos = "Subtract the sum of the digits in odd positions (1-based) from your running total."

  sequence_re = re.compile(r'((\d\s)+\d)', flags=re.IGNORECASE)
  all_steps_re = re.compile(r'Step \d\:.*', flags=re.IGNORECASE)
  strip_step_prefix = re.compile(r'Step\s*\d\:\s*', flags=re.IGNORECASE)

  try:
      sequence = re.search(sequence_re, question)
      digits = []

      if sequence:
          sequence = sequence.group()
          digits = [int(d.strip()) for d in sequence.split()]
      else:
          print(f"Cannot find sequence: {question}")


      steps = re.findall(all_steps_re, question)

      #print(f"{q}: {steps}")

      steps = [re.sub(strip_step_prefix, "", s) for s in steps]

      total = 0

      for i, s in enumerate(steps):
          if s.lower() == add_div_by_3.lower():
              temp = sum(d for d in digits if d % 3 == 0)
              if enable_debug: print(f"Step {i+1}: add_div_by_3 = {total} + {temp}")
              total += temp

          elif s.lower() == add_even.lower():
              temp = sum(d for d in digits if d % 2 == 0)
              if enable_debug: print(f"Step {i+1}: add_even = {total} + {temp}")
              total += temp

          elif s.lower() == add_count_gt_5.lower():
              temp = sum(1 for d in digits if d > 5)
              if enable_debug: print(f"Step {i+1}: add_count_gt_5 = {total} + {temp}")
              total += temp

          elif s.lower() == add_second_largest_unique.lower():
              temp = 0
              unique_digits = sorted(set(digits), reverse=True)
              if enable_debug: print(f"unique_digits: {unique_digits}")
              if len(unique_digits) > 1:
                  temp = unique_digits[1]

              if enable_debug: print(f"Step {i+1}: add_second_largest_unique = {total} + {temp}")
              total += temp

          elif s.lower() == add_sum_of_squares_even_0based.lower(): # 0-based means sequence index starts at 0
              temp = sum(digits[i]**2 for i in range(0, len(digits), 2))
              if enable_debug: print(f"Step {i+1}: add_sum_of_squares_even_0based = {total} + {temp}")
              total += temp

          elif s.lower() == add_sum_of_prime.lower():
              temp = sum(d for d in digits if d in primes)
              if enable_debug: print(f"Step {i+1}: add_sum_of_prime = {total} + {temp}")
              total += temp

          elif s.lower() == cond_even_odd_mult2_add5.lower():
              evens = sum(1 for d in digits if d % 2 == 0)
              odds = len(digits) - evens
              if evens > odds:
                  if enable_debug: print(f"Step {i+1}: cond_even_odd_mult2_add5 = {total} * 2")
                  total *= 2
              else:
                  if enable_debug: print(f"Step {i+1}: cond_even_odd_mult2_add5 = {total} + 5")
                  total += 5

          elif s.lower() == multiply_by_1_plus_count7.lower():
              temp = 1 + digits.count(7)
              if enable_debug: print(f"Step {i+1}: multiply_by_1_plus_count7 = {total} * {temp}")
              total *= temp

          elif s.lower() == multiply_by_prod_prime_pos.lower(): # 0-based
              prime_pos_digits = [digits[i] for i in primes]

              temp = 1
              for d in prime_pos_digits:
                  temp *= d

              if enable_debug: print(f"Step {i+1}: multiply_by_prod_prime_pos = {total} * {temp}")
              total *= temp

          elif s.lower() == minus_odd.lower():
              temp = sum(d for d in digits if d % 2 != 0)
              if enable_debug: print(f"Step {i+1}: minus_odd = {total} - {temp}")
              total -= temp

          elif s.lower() == minus_floor_avg.lower():
              temp = int(np.floor(np.mean(digits)))
              if enable_debug: print(f"Step {i+1}: minus_floor_avg = {total} - {temp}")
              total -= temp

          elif s.lower() == minus_sum_odd_pos.lower(): # 1-based means sequence index starts at 1
              temp = sum(digits[i] for i in range(len(digits)) if i % 2 == 0)
              if enable_debug: print(f"Step {i+1}: minus_sum_odd_pos = {total} - {temp}")
              total -= temp

          else:
              raise ValueError(f"Unsupported step {i+1}: {question}\n{s}")

      result = total
  except Exception as e:
    print(f"{question}: {e}")
    raise Exception(e)

  return (result, duration, "\n".join(steps))

# Functions: Next Number Solver

In [12]:
def cognitive_next_number(question):
  enable_debug = False

  duration = 0.0

  try:
      ## Parse the question to extract sequence of numbers
      sequence = question.split('?')[1].strip()

      numbers = [int(num) for num in sequence.split(',') if num.strip()]

      if(len(numbers) < 4):
          print(f"Too short, likely a parsing error: {sequence} != {numbers}")

      result = None
      math_type = None

      if not result:
          ## Check for Arithmetic Sequence (consistent differences between elements)
          diffs = [numbers[i+1] - numbers[i] for i in range(len(numbers)-1)]

          if enable_debug: print(f"{numbers}; {diffs}")

          if (len(diffs) > 2
              and all(d == diffs[0] for d in diffs)):
              result = numbers[-1] + diffs[0]

              math_type = "Arithmetic Sequence"

              if enable_debug: print(f"Arithmetic Sequence, {numbers}; {diffs}; {result}")

      if not result:
          ## Check for Alternating Arithmetic Sequence
          evens = [numbers[i] for i in range(len(numbers)) if i % 2 == 0]
          odds  = [numbers[i] for i in range(len(numbers)) if i % 2 != 0]

          even_diffs = [evens[i+1] - evens[i] for i in range(len(evens)-1)]
          odd_diffs  = [odds[i+1]  - odds[i]  for i in range(len(odds)-1)]

          if enable_debug: print(f"{numbers}; {evens}; {even_diffs}; {odds}; {odd_diffs};")

          if ((len(even_diffs) > 1)
              and (len(odd_diffs) > 1)
              and all(d == even_diffs[0] for d in even_diffs)
              and all(d == odd_diffs[0]  for d in odd_diffs)):

              is_even_len = (len(numbers) % 2 == 0)

              if is_even_len:
                  result = numbers[-2] + even_diffs[0]
              else:
                  result = numbers[-2] + odd_diffs[0]

              math_type = "Alt Arithmetic Sequence"

              if enable_debug: print(f"Alternating Arithmetic Sequence, {numbers}; {evens}; {even_diffs}; {odds}; {odd_diffs}; {result}")

      if not result:
          ## Fibonacci sequence (add previous term)
          is_sum_of_previous = all((numbers[i] + numbers[i-1])== numbers[i+1] for i in range(1, len(numbers)-1))

          if is_sum_of_previous:
              result = numbers[-1] + numbers[-2]
              result = numbers[-1] + result # 2nd next number

              if enable_debug: print(f"Fibonacci Sequence, {sequence}: {is_sum_of_previous}")

              math_type = "Fibonacci Sequence"

      if not result:
          if all(n != 0 for n in numbers):
              ## Check for Geometric Sequence (consistent ratios between non-zero elements)
              ratios = [numbers[i+1] / numbers[i] for i in range(len(numbers)-1)]

              if enable_debug: print(f"{numbers}; {ratios}")

              if (len(ratios) > 2
                  and all(r == ratios[0] for r in ratios)):

                  result = numbers[-1] * ratios[0]

                  if len(numbers) < 5:
                    result = result * ratios[0] # 2nd next number

                  if enable_debug: print(f"Geometric Sequence, {numbers}; {ratios}; {result}")

                  math_type = "Geometric Sequence"

      if not result:
          ## Check for Geometric Sequence amoung differences (consistent ratios between differences)
          diffs = [numbers[i+1] - numbers[i] for i in range(len(numbers)-1)]

          ratios = [diffs[i+1] / diffs[i] for i in range(len(diffs)-1)]

          if enable_debug: print(f"{diffs}: {ratios}")

          if (len(ratios) > 0
              and all(r == ratios[0] for r in ratios)):

              result = numbers[-1] + (diffs[-1] * ratios[0])

              if enable_debug: print(f"Diff Geometric Sequence, {numbers}; {ratios}; {result}")

              math_type = "Diff Geometric Sequence"

  except Exception as e:
    print(f"{question}: {e}")
    raise Exception(e)

  return (result, duration, math_type)

# Functions: Unscramble Solver

In [13]:
def sort_letters(phrase):
  letter = list(phrase.replace(" ", "").lower())

  letter.sort()

  result = "".join(letter)

  return result

def cognitive_unscramble(question):
  enable_debug = False

  answer = ""
  duration = 0.0
  prompt = ""

  scrambled_phrase = question.split(':')[1].strip()

  sorted_letters = sort_letters(scrambled_phrase)

  if enable_debug: print(f"{scrambled_phrase}, {sorted_letters}")

  if sorted_letters in unscramble_dict['Phrase']:
    answer = unscramble_dict['Phrase'][sorted_letters]
    prompt = sorted_letters
  else:
    print(f"(Not found in unscramble dictionary: orig={scrambled_phrase}, sorted={sorted_letters})")
    user_prompt = question.strip()

    user_prompt += "\n\nOnly respond with the answer."

    messages = [{
        "role": "user",
        "content": f"{user_prompt}"
    }]

    (answer, duration, prompt) = llm_api(openai_llm_gpt_45, messages)

    answer = answer.strip().lower()

    # Add answer to dictionary
    if answer.strip():
      unscramble_dict['Phrase'][sorted_letters] = answer
    else:
      print(f"Empty response from LLM: orig={scrambled_phrase}")

  return (answer, duration, prompt)

# Functions: Job Application

In [14]:
def answer_job_application(r):
  question = r.Question.strip()
  question_item_type = r.Item.strip()
  question_format = r.Format.strip()

  answer = ""
  duration = 0.0
  prompt = ""

  if (question_item_type.lower().startswith(resume_item_prefix.lower())):

     (answer, duration, prompt) = skills_resume_text(r)

  elif (question_item_type.lower().startswith(interview_item_prefix.lower())):

     (answer, duration, prompt) = skills_interview_text(r)

  elif (question_item_type.lower().startswith(cognitive_item_prefix.lower())
        and question_format.lower().startswith("Integer".lower())
        and question.lower().startswith("Below is a random sequence of digits:".lower())):

     (answer, duration, prompt) = cognitive_numeric_ops(question)

  elif (question_item_type.lower().startswith(cognitive_item_prefix.lower())
        and question_format.lower().startswith("Integer".lower())
        and question.lower().startswith("What is the next number?".lower())):

     (answer, duration, prompt) = cognitive_next_number(question)

  elif (question_item_type.lower().startswith(cognitive_item_prefix.lower())
        and question_format.lower().startswith("Text".lower())):

     (answer, duration, prompt) = cognitive_unscramble(question)

  elif (question_item_type.lower().startswith(personality_likert_item_prefix.lower())):

    (answer, duration, prompt) = personality_likert_rating(r)

  elif (question_item_type.lower().startswith(personality_choice_item_prefix.lower())):

    (answer, duration, prompt) = personality_choice_rating(r)

  elif (question_item_type.lower().startswith(job_readiness_item_prefix.lower())):

    (answer, duration, prompt) = skills_job_readiness_rating(r)

  else:

    raise ValueError(f"Unsupported question type: {question_item_type}")

  return (answer, duration, prompt)

# Preprocessing: Merge Raw Datasets

In [15]:
print("Merge raw data datasets...")

# Load Job Listing Dataset
df_job_listings = pd.read_excel(job_listings_path, dtype=str)

# Generate Management Level
df_job_listings[management_col] = df_job_listings[role_title_col].apply(lambda j: gen_mangagement_level(j))

# Generate Company Size
df_job_listings[company_size_col] = df_job_listings[employee_count_col].apply(lambda e: gen_company_size(e))

# Load Job Application Dataset
df_job_applications = pd.read_excel(job_applications_path, dtype=str)

# Merge Job Datasets
df_combined = pd.merge(df_job_applications, df_job_listings, left_on=job_title_col, right_on=role_title_col, how='left')
df_combined.drop(columns=[role_title_col], axis=1, inplace=True)
df_combined.index = df_combined.index + 1
df_combined.rename_axis(id_col, inplace=True)
#df_combined.to_excel(combined_path)

print("Completed.")

Merge raw data datasets...
Completed.


# Preprocessing: Skills by Job

In [16]:
print("Process skills by job...")

# Extract Job Readiness Questions
df_job_all_skills = df_combined[(df_combined[item_col].str.startswith(job_readiness_item_prefix))].copy()

# Extract Job Readiness skill (strip out question)
df_job_all_skills[question_col] = df_job_all_skills[question_col].str.replace(job_readiness_question_prefix, "")
df_job_all_skills[question_col] = df_job_all_skills[question_col].str.strip()

# Join all skills as csv per job
df_job_all_skills[skills_col] = df_job_all_skills.groupby([job_title_col])[question_col].transform(lambda x: '; '.join(x))

# Extract unique job skills
df_job_skills_unique = df_job_all_skills.groupby([question_col])[question_col].count().reset_index(name=count_col)

df_job_skills_unique.to_excel(job_skills_unique_path, index=False)

# Keep only unique jobs
df_job_skills = df_job_all_skills[job_info_cols].copy()
df_job_skills.drop_duplicates(inplace=True)
df_job_skills.reset_index(drop=True, inplace=True)
df_job_skills.index = df_job_skills.index + 1
df_job_skills.rename_axis(id_col, inplace=True)

df_job_skills.to_excel(job_skills_path)

df_job_skills_answers = df_job_skills.copy()
df_job_skills_answers[response_col] = ""
df_job_skills_answers[duration_col] = ""
df_job_skills_answers[prompt_col] = ""

if os.path.isfile(job_skills_answers_path):
  df_job_skills_answers = pd.read_excel(job_skills_answers_path, index_col=0)

print(f"Start Processing: ", datetime.now())
print("Progress: ", end="")
for i, r in df_job_skills_answers.iterrows():
  if i%10 == 0:
      print(".", end="")

  response = str(r.Response).strip().lower()

  if (len(response)<1 or response=="nan"):
    (answer, duration, prompt) = skills_job_readiness_llm(r)

    df_job_skills_answers.loc[df_job_skills_answers.index == i, [response_col]] = answer
    df_job_skills_answers.loc[df_job_skills_answers.index == i, [duration_col]] = duration
    df_job_skills_answers.loc[df_job_skills_answers.index == i, [prompt_col]] = prompt

    if i>0 and i%50==0:
        print("\nSaving Progress: ", i, datetime.now())
        df_job_skills_answers.to_excel(job_skills_answers_path)

print(f"\nEnd Processing: ", datetime.now())
df_job_skills_answers.to_excel(job_skills_answers_path)

print("Completed.")

Process skills by job...
Start Processing:  2025-04-10 22:23:52.307323
Progress: 
End Processing:  2025-04-10 22:23:58.157110
Completed.


# Preprocessing: Interview Questions

In [17]:
print("Process interview questions...")

# Extract Interview Questions
df_interview = df_combined[(df_combined[item_col].str.startswith(interview_item_prefix))].copy()

# Keep only unique cases
df_interview = df_interview[[question_col]].copy()
df_interview.drop_duplicates(inplace=True)
df_interview.reset_index(drop=True, inplace=True)
df_interview.index = df_interview.index + 1
df_interview.rename_axis(id_col, inplace=True)

df_interview.to_excel(interview_path)

df_interview_answers = df_interview.copy()
df_interview_answers[response_col] = ""
df_interview_answers[duration_col] = ""
df_interview_answers[prompt_col] = ""

if os.path.isfile(interview_answers_path):
  df_interview_answers = pd.read_excel(interview_answers_path, index_col=0)

print(f"Start Processing: ", datetime.now())
print("Progress: ", end="")
for i, r in df_interview_answers.iterrows():
  if i%10 == 0:
      print(".", end="")

  response = str(r.Response).strip().lower()

  if (len(response)<1 or response=="nan"):
    (answer, duration, prompt) = skills_interview_llm(r)

    df_interview_answers.loc[df_interview_answers.index == i, [response_col]] = answer
    df_interview_answers.loc[df_interview_answers.index == i, [duration_col]] = duration
    df_interview_answers.loc[df_interview_answers.index == i, [prompt_col]] = prompt

    if i>0 and i%50==0:
        print("\nSaving Progress: ", i, datetime.now())
        df_interview_answers.to_excel(interview_answers_path)

print(f"\nEnd Processing: ", datetime.now())
df_interview_answers.to_excel(interview_answers_path)

print("Completed.")

Process interview questions...
Start Processing:  2025-04-10 22:23:58.254961
Progress: .
End Processing:  2025-04-10 22:24:50.006122
Completed.


# Preprocessing: Personality Likert

In [18]:
print("Process personality likert questions...")

# Extract Personality Likert Questions
df_personality_likert = df_combined[(df_combined[item_col].str.startswith(personality_likert_item_prefix))].copy()

# Keep only unique cases
df_personality_likert = df_personality_likert[[question_col]].copy()
df_personality_likert.drop_duplicates(inplace=True)
df_personality_likert.reset_index(drop=True, inplace=True)
df_personality_likert.index = df_personality_likert.index + 1
df_personality_likert.rename_axis(id_col, inplace=True)

df_personality_likert.to_excel(personality_likert_path)

df_personality_likert_answers = df_personality_likert.copy()
df_personality_likert_answers[response_col] = ""
df_personality_likert_answers[duration_col] = ""
df_personality_likert_answers[prompt_col] = ""

if os.path.isfile(personality_likert_answers_path):
  df_personality_likert_answers = pd.read_excel(personality_likert_answers_path, index_col=0)

print(f"Start Processing: ", datetime.now())
print("Progress: ", end="")
for i, r in df_personality_likert_answers.iterrows():
  if i%10 == 0:
      print(".", end="")

  response = str(r.Response).strip().lower()

  if (len(response)<1 or response=="nan"):
    (answer, duration, prompt) = personality_likert_llm(r)

    df_personality_likert_answers.loc[df_personality_likert_answers.index == i, [response_col]] = answer
    df_personality_likert_answers.loc[df_personality_likert_answers.index == i, [duration_col]] = duration
    df_personality_likert_answers.loc[df_personality_likert_answers.index == i, [prompt_col]] = prompt

    if i>0 and i%50==0:
        print("\nSaving Progress: ", i, datetime.now())
        df_personality_likert_answers.to_excel(personality_likert_answers_path)

print(f"\nEnd Processing: ", datetime.now())
df_personality_likert_answers.to_excel(personality_likert_answers_path)

print("Completed.")

Process personality likert questions...
Start Processing:  2025-04-10 22:24:50.046143
Progress: .....
Saving Progress:  50 2025-04-10 22:25:16.979845
.....
Saving Progress:  100 2025-04-10 22:25:42.741419
..
End Processing:  2025-04-10 22:25:54.684609
Completed.


# Preprocessing: Personality Choice

In [19]:
print("Process personality choice questions...")

# Extract Personality Choice Questions
df_personality_choice = df_combined[(df_combined[item_col].str.startswith(personality_choice_item_prefix))].copy()

# Keep only unique cases
df_personality_choice = df_personality_choice[[question_col]].copy()
df_personality_choice.drop_duplicates(inplace=True)
df_personality_choice.reset_index(drop=True, inplace=True)
df_personality_choice.index = df_personality_choice.index + 1
df_personality_choice.rename_axis(id_col, inplace=True)

df_personality_choice.to_excel(personality_choice_path)

df_personality_choice_answers = df_personality_choice.copy()
df_personality_choice_answers[response_col] = ""
df_personality_choice_answers[duration_col] = ""
df_personality_choice_answers[prompt_col] = ""

if os.path.isfile(personality_choice_answers_path):
  df_personality_choice_answers = pd.read_excel(personality_choice_answers_path, index_col=0)

print(f"Start Processing: ", datetime.now())
print("Progress: ", end="")

messages = []

formatted_system_prompt = personality_choice_system_prompt + "\n\n" + personality_choice_question_prefix

messages.append({
    "role": "system",
    "content": f"{formatted_system_prompt}"
})

for i, r in df_personality_choice_answers.iterrows():
  if i%10 == 0:
      print(".", end="")

  user_prompt = r.Question.replace(personality_choice_question_prefix, "").strip()

  messages.append({
      "role": "user",
      "content": f"{user_prompt}"
  })

  response = str(r.Response).strip().lower()

  if (len(response)<1 or response=="nan"):
    (answer, duration, prompt) = personality_choice_llm(r, messages)

    df_personality_choice_answers.loc[df_personality_choice_answers.index == i, [response_col]] = answer
    df_personality_choice_answers.loc[df_personality_choice_answers.index == i, [duration_col]] = duration
    df_personality_choice_answers.loc[df_personality_choice_answers.index == i, [prompt_col]] = prompt

    if i>0 and i%50==0:
        print("\nSaving Progress: ", i, datetime.now())
        df_personality_choice_answers.to_excel(personality_choice_answers_path)
  else:
    answer = r.Response

  messages.append({
      "role": "assistant",
      "content": f"{answer}"
  })

print(f"\nEnd Processing: ", datetime.now())
df_personality_choice_answers.to_excel(personality_choice_answers_path)

print("Completed.")

Process personality choice questions...
Start Processing:  2025-04-10 22:25:54.732247
Progress: .....
Saving Progress:  50 2025-04-10 22:29:30.525290

End Processing:  2025-04-10 22:29:34.567570
Completed.


# Generate Answers: All

In [20]:
print("Consolidate all answers...")

df_answers = df_combined.copy()

stop_at_index = None

print(f"Start Processing: ", datetime.now())
try:
  for i, r in df_answers.iterrows():
      if i%1000 == 0:
          print(".", end="")

      (answer, duration, prompt) = answer_job_application(r)

      df_answers.loc[df_answers.index == i, [response_col]] = answer
      df_answers.loc[df_answers.index == i, [duration_col]] = duration
      df_answers.loc[df_answers.index == i, [prompt_col]] = prompt

      if i>0 and i%5000==0:
          print("\nSaving Progress: ", i, datetime.now())
          df_answers.to_excel(answers_path)

      if stop_at_index and i>=stop_at_index:
          print("\nStop: ", i, datetime.now())
          break
except Exception as e:
  print(f"Id {i}: {e}")
  raise Exception(e)

print(f"\nEnd Processing: ", datetime.now())

df_answers.to_excel(answers_path)

print("Completed.")

Consolidate all answers...
Start Processing:  2025-04-10 22:29:34.604165

End Processing:  2025-04-10 22:29:36.372167
Completed.


# Submission Prep

In [21]:
print("Prepare submission file...")

# Save Updates to Unscramble Dictionary
df_unscramble = pd.DataFrame.from_dict(unscramble_dict)
df_unscramble.rename_axis(id_col, inplace=True)
df_unscramble.to_excel(unscramble_lookup_path)

# Create Submission File
df_answers.sort_values(id_col, inplace=True)

df_submit = df_answers[submit_keep_cols].copy()
df_submit.to_excel(submit_path, index=False)

# Check Submission File for Missing Responses
validate_submission(df_submit)

print("Completed.")

Prepare submission file...
Completed.
